# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "demo_coding_vs_intergenomic_seqs"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_coding_vs_intergenomic_seqs 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

2021-11-23 00:57:12.264395: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-23 00:57:12.264411: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-11-23 00:57:13.569777: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-23 00:57:13.569796: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-11-23 00:57:13.570045: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `demo_coding_vs_intergenomic_seqs` has 2 classes: coding_seqs, intergenomic_seqs.

All lenghts of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


,train,test
coding_seqs,37500,12500
intergenomic_seqs,37500,12500


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 75000 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
# vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-11-23 00:57:18.254473: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 'a', 't', 'g', 'c']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


   1/1172 [..............................] - ETA: 22:10 - loss: 0.8596 - binary_accuracy: 0.4688 - f1_score: 0.0606

   6/1172 [..............................] - ETA: 14s - loss: 0.7279 - binary_accuracy: 0.5521 - f1_score: 0.1373  

  11/1172 [..............................] - ETA: 14s - loss: 0.6342 - binary_accuracy: 0.6307 - f1_score: 0.3674

  15/1172 [..............................] - ETA: 14s - loss: 0.5992 - binary_accuracy: 0.6656 - f1_score: 0.4931

...

1162/1172 [============================>.] - ETA: 0s - loss: 0.2363 - binary_accuracy: 0.9047 - f1_score: 0.8958

1165/1172 [============================>.] - ETA: 0s - loss: 0.2365 - binary_accuracy: 0.9047 - f1_score: 0.8958

1168/1172 [============================>.] - ETA: 0s - loss: 0.2364 - binary_accuracy: 0.9047 - f1_score: 0.8958

1171/1172 [============================>.] - ETA: 0s - loss: 0.2364 - binary_accuracy: 0.9046 - f1_score: 0.8958

1172/1172 [==============================] - 19s 16ms/step - loss: 0.2365 - binary_accuracy: 0.9046 - f1_score: 0.8958


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 25000 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

  1/391 [..............................] - ETA: 2:08 - loss: 0.1955 - binary_accuracy: 0.8906 - f1_score: 0.9355

  8/391 [..............................] - ETA: 2s - loss: 0.2896 - binary_accuracy: 0.8887 - f1_score: 0.8934  

 14/391 [>.............................] - ETA: 3s - loss: 0.2698 - binary_accuracy: 0.8917 - f1_score: 0.8932

 20/391 [>.............................] - ETA: 3s - loss: 0.2740 - binary_accuracy: 0.8891 - f1_score: 0.8934

 25/391 [>.............................] - ETA: 3s - loss: 0.2837 - binary_accuracy: 0.8844 - f1_score: 0.8867

...

369/391 [===========================>..] - ETA: 0s - loss: 0.2636 - binary_accuracy: 0.8944 - f1_score: 0.8927

378/391 [============================>.] - ETA: 0s - loss: 0.2644 - binary_accuracy: 0.8947 - f1_score: 0.8926

386/391 [============================>.] - ETA: 0s - loss: 0.2647 - binary_accuracy: 0.8946 - f1_score: 0.8928

391/391 [==============================] - 3s 7ms/step - loss: 0.2643 - binary_accuracy: 0.8947 - f1_score: 0.8930


[0.26428142189979553, 0.8946800231933594, 0.892976701259613]